**Importing the Dependencies**

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Loading the Dataset**

In [34]:
data = pd.read_csv("/content/IMDB Dataset.csv")


In [35]:
data.shape

(50000, 2)

In [36]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [37]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [38]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [39]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [40]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [41]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [42]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [43]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [44]:
# Tokenize text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [45]:
print(X_train)

[[2946 3749 1828 ...  205  351 3856]
 [   3 4004  208 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  103  125 7285]
 [   0    0    0 ...   70   73 2062]]


In [46]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [ 133    6  117 ...    7 9050 8494]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [47]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [48]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**LSTM - Long Short-Term Memory**

In [49]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1411713 (5.39 MB)
Trainable params: 1411713 (5.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Training the Model**

In [52]:
model.fit(X_train, Y_train, epochs=5, batch_size=128, validation_split=0.2)

Epoch 1/5
250/250 [==============================] - 121s 462ms/step - loss: 0.4225 - accuracy: 0.8012 - val_loss: 0.3176 - val_accuracy: 0.8692
Epoch 2/5
250/250 [==============================] - 104s 417ms/step - loss: 0.2407 - accuracy: 0.9050 - val_loss: 0.3026 - val_accuracy: 0.8830
Epoch 3/5
250/250 [==============================] - 100s 399ms/step - loss: 0.1864 - accuracy: 0.9301 - val_loss: 0.3185 - val_accuracy: 0.8671
Epoch 4/5
250/250 [==============================] - 101s 402ms/step - loss: 0.1482 - accuracy: 0.9448 - val_loss: 0.3269 - val_accuracy: 0.8749
Epoch 5/5
250/250 [==============================] - 96s 384ms/step - loss: 0.1204 - accuracy: 0.9567 - val_loss: 0.4031 - val_accuracy: 0.8597


**Model Evaluation**

In [53]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 [==============================] - 18s 58ms/step - loss: 0.3983 - accuracy: 0.8639
Test Loss: 0.3983314633369446
Test Accuracy: 0.8639000058174133


**Building a Predictive System**

In [54]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [55]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 275ms/step
The sentiment of the review is: positive


In [56]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 63ms/step
The sentiment of the review is: negative


In [57]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 64ms/step
The sentiment of the review is: negative


In [58]:
# example usage
new_review = "This movie was horrible that none one could even resist it for 1 second."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 66ms/step
The sentiment of the review is: negative
